In [1]:
%run util.py

In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
with open(data_folder + 'jam_ref_data.json', 'r') as json_file:
    jam_ref_data = json.load(json_file)

with open(data_folder + 'jam_test_data.json', 'r') as json_file:
    jam_test_data = json.load(json_file)

In [5]:
# number of jams in reference data
print('number of jams in ref data: \n')
print(len(jam_ref_data))

# number of jams in test data
print('number of jams in test data: \n')
print(len(jam_test_data))

number of jams in ref data: 

3218
number of jams in test data: 

3882


In [6]:
[distToClusterCenter, loc_label, loc_centroid] = zload(data_folder + 'loc_quant_results.pkz')
[time_range, KL_list, etaWC_list, win_jam] = zload(data_folder + 'detec_results.pkz')
para_dict = zload(data_folder + 'para_dict.pkz')

In [7]:
loc_centroid

[[-71.111306,
  42.364303,
  -71.111306,
  42.366344,
  -71.104657,
  42.364303,
  -71.104657,
  42.366344],
 [-71.110251,
  42.335729,
  -71.110251,
  42.338486,
  -71.106477,
  42.335729,
  -71.106477,
  42.338486],
 [-71.085217,
  42.344783,
  -71.085217,
  42.347836,
  -71.08297,
  42.344783,
  -71.08297,
  42.347836]]

In [8]:
latitudes = []

for i in range(3):
    for j in range(4):
        latitudes.append(loc_centroid[i][2*j+1])

In [9]:
longitudes = []

for i in range(3):
    for j in range(4):
        longitudes.append(loc_centroid[i][2*j])

In [10]:
import gmplot

gmap = gmplot.GoogleMapPlotter(42.351848, -71.110730, 13)

gmap.scatter([42.351848], [-71.110730], c='g', marker=True)

gmap.circle(42.351848, -71.110730, 3000, '#ffefef', ew=4)

gmap.scatter(latitudes, longitudes, c='r', marker=True)

gmap.draw('./loc_clusters.html')

In [11]:
len(time_range), len(KL_list), len(etaWC_list), time_range[0:2]

(286, 286, 286, [0.0, 5.0])

In [12]:
win_size = para_dict['win_size']

alarmWinIdxList = []
alarmWinIdxStartTimeDict = {}
potential_ano_list = []
for i in range(len(KL_list)):
    if KL_list[i] > etaWC_list[i]:
        alarmWinIdxList.append(i)
        alarmWinIdxStartTimeDict[i] = 60 * time_range[i] + win_size
        for j in range(len(win_jam[str(i)])):
            potential_ano_list.append(win_jam[str(i)][j])

In [13]:
alarmWinIdxStartTimeDict, alarmWinIdxList

({148: 45000.0,
  149: 45300.0,
  170: 51600.0,
  171: 51900.0,
  256: 77400.0,
  257: 77700.0,
  262: 79200.0,
  263: 79500.0,
  274: 82800.0,
  275: 83100.0},
 [148, 149, 170, 171, 256, 257, 262, 263, 274, 275])

In [14]:
with open(data_folder + 'sample_path_jam_test_data_sorted.json', 'r') as json_file:
    sample_path_jam_test_data_sorted = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_test_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_ref_data_sorted.json', 'r') as json_file:
    sample_path_jam_ref_data_sorted = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_ref_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_ref_data_sorted_trans = json.load(json_file)

In [15]:
lengthTestList = []
for i in range(len(sample_path_jam_ref_data_sorted)):
    lengthTestList.append(sample_path_jam_ref_data_sorted[str(i)]['length'])

In [16]:
mu_length = np.mean(lengthTestList)
sigma_length = np.std(lengthTestList)

# apply the three-sigma rule
thres_length = mu_length + 3 * sigma_length

In [17]:
thres_length

1921.1039361100393

In [18]:
# len(set(potential_ano_list)), set(potential_ano_list)

In [19]:
len(sample_path_jam_test_data_sorted_trans)

3881

In [20]:
# Refine the detection results; if a jam's length is larger than thres_length,
# then the jam is "indeed" abnormal.
ano_list = []
for key in set(potential_ano_list):
    if (sample_path_jam_test_data_sorted[key]['length'] > thres_length):
        ano_list.append(key)

In [21]:
len(ano_list), ano_list

(4, ['788', '3005', '742', '3657'])

In [22]:
ano_dict = {'idx_' + str(i):int(ano_list[i]) for i in range(len(ano_list))}

In [23]:
ano_dict

{'idx_0': 788, 'idx_1': 3005, 'idx_2': 742, 'idx_3': 3657}

In [24]:
import scipy.io

scipy.io.savemat('./anoDict.mat', ano_dict)

In [25]:
anoStartTimeDict = {}
for key in ano_list:
    anoStartTimeDict[key] = sample_path_jam_test_data_sorted[key]['startTime'] / 60.0

In [26]:
anoStartTimeDict

{'3005': 1285.0039599999998,
 '3657': 1375.0031683333334,
 '742': 745.0015683333334,
 '788': 745.005035}

In [27]:
anoFirstAlarmTimeDict = {}
for key in ano_list:
    for winIdx in alarmWinIdxList:
        if key in win_jam[str(winIdx)]:
            anoFirstAlarmTimeDict[key] = alarmWinIdxStartTimeDict[winIdx] / 60.0
            break

In [28]:
sample_path_jam_test_data_sorted['739']['startTime'], sample_path_jam_test_data_sorted['792']['startTime']

(44700.0239, 44700.3521)

In [29]:
anoFirstAlarmTimeDict

{'3005': 1290.0, '3657': 1380.0, '742': 750.0, '788': 750.0}

In [30]:
alarmWinIdxStartTimeDict

{148: 45000.0,
 149: 45300.0,
 170: 51600.0,
 171: 51900.0,
 256: 77400.0,
 257: 77700.0,
 262: 79200.0,
 263: 79500.0,
 274: 82800.0,
 275: 83100.0}

In [31]:
for key in ano_list:
    print(sample_path_jam_test_data_sorted[key])

{u'uuid': u'a03907a1-0ccb-3c22-90a8-4fc7f118537d', u'numPts': 36.0, u'length': 1962.0, u'startTime': 44700.3021, u'alpha': 1.0, u'speed': 7.269444444444445}
{u'uuid': u'949eca59-4be7-3352-af0e-959032a35a2c', u'numPts': 25.0, u'length': 2731.0, u'startTime': 77100.2376, u'alpha': 1.0, u'speed': 12.947222222222221}
{u'uuid': u'0883bcf6-5580-3522-bfaa-13dcda693c43', u'numPts': 26.0, u'length': 6874.0, u'startTime': 44700.0941, u'alpha': 5.0, u'speed': 8.81111111111111}
{u'uuid': u'949eca59-4be7-3352-af0e-959032a35a2c', u'numPts': 25.0, u'length': 3568.0, u'startTime': 82500.1901, u'alpha': 1.0, u'speed': 8.919444444444444}


### run the following cells after obtaining matched alerts in the 11th notebook

In [32]:
with open('./filteredAlerts.json', 'r') as json_file:
    filteredAlerts = json.load(json_file)

In [33]:
filteredAlerts

{u'idx_2950': {u'alertType': u'JAM',
  u'endTime': 75960.0,
  u'latitude': u'42.359444',
  u'longitude': u'-71.120531',
  u'startTime': 75900.0},
 u'idx_2951': {u'alertType': u'JAM',
  u'endTime': 75960.0,
  u'latitude': u'42.360081',
  u'longitude': u'-71.119965',
  u'startTime': 75900.0},
 u'idx_3030': {u'alertType': u'JAM',
  u'endTime': 79560.0,
  u'latitude': u'42.34357',
  u'longitude': u'-71.087383',
  u'startTime': 79500.0},
 u'idx_3031': {u'alertType': u'JAM',
  u'endTime': 79560.0,
  u'latitude': u'42.37798',
  u'longitude': u'-71.107817',
  u'startTime': 79500.0},
 u'idx_3032': {u'alertType': u'JAM',
  u'endTime': 79560.0,
  u'latitude': u'42.371509',
  u'longitude': u'-71.088811',
  u'startTime': 79500.0},
 u'idx_3033': {u'alertType': u'WEATHERHAZARD',
  u'endTime': 79560.0,
  u'latitude': u'42.331925',
  u'longitude': u'-71.117927',
  u'startTime': 79500.0},
 u'idx_3034': {u'alertType': u'JAM',
  u'endTime': 79560.0,
  u'latitude': u'42.354571',
  u'longitude': u'-71.11095

In [34]:
for key in filteredAlerts.keys():
    if (filteredAlerts[key]['latitude'] == '42.349125' and filteredAlerts[key]['longitude'] == '-71.10778' \
        and filteredAlerts[key]['startTime'] == 77100):
        print(key)
        print(filteredAlerts[key])

idx_3116
{u'latitude': u'42.349125', u'alertType': u'JAM', u'longitude': u'-71.10778', u'startTime': 77100.0, u'endTime': 77160.0}


In [35]:
for key in filteredAlerts.keys():
    if (filteredAlerts[key]['latitude'] == '42.361951' and filteredAlerts[key]['longitude'] == '-71.117963' \
        and filteredAlerts[key]['startTime'] == 44700):
        print(key)
        print(filteredAlerts[key])

idx_928
{u'latitude': u'42.361951', u'alertType': u'WEATHERHAZARD', u'longitude': u'-71.117963', u'startTime': 44700.0, u'endTime': 44760.0}


In [36]:
for key in filteredAlerts.keys():
    if (filteredAlerts[key]['latitude'] == '42.355691' and filteredAlerts[key]['longitude'] == '-71.119172' \
        and filteredAlerts[key]['startTime'] == 82500):
        print(key)
        print(filteredAlerts[key])

idx_3460
{u'latitude': u'42.355691', u'alertType': u'JAM', u'longitude': u'-71.119172', u'startTime': 82500.0, u'endTime': 82560.0}


In [37]:
for key in filteredAlerts.keys():
    if (filteredAlerts[key]['latitude'] == '42.353782' and filteredAlerts[key]['longitude'] == '-71.117442' \
        and filteredAlerts[key]['startTime'] == 44700):
        print(key)
        print(filteredAlerts[key])

idx_1062
{u'latitude': u'42.353782', u'alertType': u'JAM', u'longitude': u'-71.117442', u'startTime': 44700.0, u'endTime': 44760.0}


In [38]:
77100.0/3600, 44700.0/3600, 82500.0/3600

(21.416666666666668, 12.416666666666666, 22.916666666666668)